# Item 38. 연산 또는 액션을 전달할 때는 인터페이스 대신 함수 타입을 사용해라

## 메서드가 하나만 있는 인터페이스, 함수형 인터페이스, SAM

- 자바와 같은 저수준 언어는 함수 타입이 존재하지 않음
- 대신 메서드 하나만 있는 인터페이스를 사용하여 함수를 전달
- @FunctionalInterface 어노테이션을 사용하여 함수형 인터페이스임을 명시
- SAM(Single Abstract Method) 인터페이스라고도 함

``` java
@FunctionalInterface
interface FuncInterface {
    void abstractFun(int x);
}

class Test {
    
    public void test(FuncInterface fob) {
        fob.abstractFun(5);
    }
    public static void main(String[] args) {
        // 익명 클래스로 구현하기
        Test t = new Test();
        t.test(new FuncInterface() {
            @Override
            public void abstractFun(int x) {
                System.out.println("Value of x: " + x);
            }
        });
        
        // 람다식으로 구현하기
        t.test((int x) -> System.out.println("Value of x: " + x));
        
        // 메서드 참조로 구현하기
        t.test(System.out::println);
        
    }
}
```

- 만약 특정 메서드의 파라미터가 함수형 인터페이스를 타입으로 받는다면, 이러한 인터페이스를 구현한 객체를 전달받는다는 의미
- java 8 이후론 람다식을 사용하여 함수형 인터페이스를 구현할 수 있음


In [1]:
fun interface Runnable {
    fun run()
}

fun runTwice(r: Runnable) {
    r.run()
    r.run()
}
// 1. sam 인터페이스를 구현한 객체를 전달
runTwice(object : Runnable {
    override fun run() {
        println("Hello!")
    }
})

// 2. 람다식을 사용하여 함수형 인터페이스를 구현
runTwice {  -> println("Hello!") }



Hello!
Hello!


## 코틀린 함수 타입
- 코틀린은 함수 타입으로 파라미터를 정의하고, 람다로 정의할 수 있음

In [2]:
// 함수타입으로 정의
fun runTwiceWithFuncType(r: () -> Unit) {
    r()
    r()
}

runTwiceWithFuncType { println("Hello!") }

Hello!
Hello!


## 함수형 인터페이스로 정의하기 vs 함수 타입으로 정의 분석


In [ ]:

fun interface Test {
    fun test()
}

fun testWrapper(test: Test) {
    test.test()
}

fun testTypeWrapper(test: () -> Unit) {
    test()
}

fun main() {

    // 함수형 인터페이스시 익명 객체를 생성하여 전달 가능
    testWrapper(object : Test {
        override fun test() {
            println("test")
        }
    })

    // 함수형 인터페이스시 익명 객체를 다른타입으로 정의 불가능
//    testWrapper { object : () -> Unit  {
//        override fun test() {
//            println("test")
//        }
//    }
//

    // 함수형 인터페이스시 SAM Conversion 으로 보다 간결하게 문법사용
    testWrapper{ Test { println("test") }}

    // 함수형 인터페이스시 람다로 전달 가능
    testWrapper { println("test") }

    // 함수 타입시 익명 객체를 생성하여 전달 가능
    testTypeWrapper { object : () -> Unit {
        override fun invoke() {
            println("test")
        }
    } }

    // 함수 타입시 익명 객체를 다른타입으로 정의 가능 - 타입 인풋 아웃풋만 같으면 가능하다
    testTypeWrapper { object : Test {
        override fun test() {
            println("test")
        }
    } }
    
    // 함수 타입시 SAM Conversion 으로 함수형 인터페이스 SAM을 넘기기 가능
    testTypeWrapper { Test { println("test") } }

    // 함수 타입시 람다로 전달 가능
    testTypeWrapper { println("test") }
}


## 정리
- 위의 분석을 통해 보면 함수형 인터페이스의 장점은 SAM 변환시 조금더 강타입선언이 가능하다는 점이다.
- 하지만 결국 람다로 전달이 가능하므로 가장 간결하게 작성시엔 동일한 타입 검증 수준뿐이 하질 못한다.
- 네이밍을 지을수 있다는 장점이 존재하지만, 그만큼 코드작성이 필요하고 함수 타입역시 typealias로 선언하여 사용할 수 있다.

## 뭐가 더 나은가?
- 책에서는 함수 타입이 더 낫다고 말은 하지만, 현재 코틀린 버전에서는 함수형 인터페이스를 사용해도 모든문법들이 거의 동일하게 가능하다.
- 코드 유지보수 측면에서 타입앨리어스보다는 객체지향에 조금더 어울리는 함수형 인터페이스가 더 나은 선택아닐까?